In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from braindecode.models import EEGConformer
import pandas as pd
from collections import OrderedDict
import math
import importlib
import Attention_temporal_model
importlib.reload(Attention_temporal_model)
from Attention_temporal_model import ShallowAttentionNet
from shallow_laurits import ShallowFBCSPNet
from torchinfo import summary

In [25]:
in_chans = 22
n_classes = 4
input_window_samples = 1125
model = ShallowAttentionNet(in_chans,n_classes,input_window_samples)

AttributeError: cannot assign module before Module.__init__() call

In [ ]:
print(model)

ShallowAttentionNet(
  (spatial_att): SpatialAttention(
    (query): Linear(in_features=22, out_features=22, bias=False)
    (key): Linear(in_features=22, out_features=22, bias=False)
    (value): Linear(in_features=22, out_features=22, bias=False)
    (out_linear): Linear(in_features=22, out_features=22, bias=False)
  )
  (temporal): Conv2d(1, 10, kernel_size=(1, 25), stride=(1, 1))
  (batch_norm): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 100), stride=(1, 100), padding=0)
  (dropout): Dropout(p=0.7, inplace=False)
  (fc): LazyLinear(in_features=0, out_features=4, bias=True)
)


In [ ]:
summary(model,input_size=(2, 22, 1125),col_names=["input_size", "output_size", "num_params", "kernel_size"])

after unsqueeze: torch.Size([2, 1, 22, 1125])
torch.Size([2, 22, 1125])
torch.Size([2, 1125, 22])
torch.Size([2, 1125, 22])


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
ShallowAttentionNet                      [2, 22, 1125]             [2, 4]                    --                        --
├─SpatialAttention: 1-1                  [2, 1, 22, 1125]          [2, 1, 22, 1125]          1                         --
│    └─Linear: 2-1                       [2, 1125, 22]             [2, 1125, 22]             484                       --
│    └─Linear: 2-2                       [2, 1125, 22]             [2, 1125, 22]             484                       --
│    └─Linear: 2-3                       [2, 1125, 22]             [2, 1125, 22]             484                       --
│    └─Linear: 2-4                       [2, 1125, 22]             [2, 1125, 22]             484                       --
├─Conv2d: 1-2                            [2, 1, 22, 1125]          [2, 10, 22, 1101]         260                       [1, 25]
├─BatchNo